Part 1 of the Civil Asset Forfeiture analysis. This script takes in the raw google form responses from court-watchers and joins them into one dataset, as well as performing some cleaning. 

After this is run, the next step is to complete the cleaning manually, and then run the script for part 2, which generates summary tables and analyses. 

Last edited by Sophie Bair 10/26/22

In [ ]:
import numpy as np 
import pandas as pd
import unidecode

In [ ]:
pip install pandoc

In [ ]:
import pandoc

# 1. import & join files 

In [ ]:
caf1 = pd.read_excel('CIVIL ASSET FORFEITURE - COURT-WATCHING SURVEY (Responses).xlsx')

In [ ]:
caf1.head()

In [ ]:
caf2 = pd.read_excel('II - CIVIL ASSET FORFEITURE - COURT-WATCHING SURVEY (Responses).xlsx')

In [ ]:
caf2.head()

In [ ]:
caf = pd.concat([caf1, caf2])

In [ ]:
caf.head()

In [ ]:
#caf.columns # seeems like no dups 

In [ ]:
caf.columns = ['judge_behavior', 'name', 'fine_fee_issues', 'date', 'seized_property_type', 'litigant_race',
              'judge_name', 'fee_waiver_filed', 'hardship_exception_requested', 'case_settled_or_resolved',
              'zoom_difficulties', 'court_division_district', 'email', 'litigant_gender', 'judge_race',
              'seized_property_value', 'atty_present', 'property_description', 'admin_difficulties', 
              'affiliation', 'courtroom_number', 'judge_gender', 'litigant_understanding', 'more_than_one_asset', 
              'party', 'jointly_owned_property', 'seizing_entity', 'probable_cause_hearing', 'seizure_reason', 
              'lienholder_atty_appeared', 'innocent_owners_mention', 'cost_bond_mention', 'isp_district', 
              'litigant_understanding_explanation', 'fee_waiver_filed_explanation', 'suburban_district', 
               'jointly_owned_property_explanation', 'fine_fee_issues_explanation', 'more_than_one_asset_explanation', 
               'case_settled_or_resolved_explanation', 'probable_cause_established', 'zoom_difficulties_explanation', 
              'admin_difficulties_explanation', 'comments_concerns', 'party_explanation', 'atty_present_explanation', 
              'timestamp', 'overall_narrative']

In [ ]:
caf.head()

In [ ]:
caf.dtypes

In [ ]:
# create index to uniquely identify cases 
caf['index_created'] = range(0, len(caf))

# 2. Clean demographic/basic case info 

2a. court-watchers 

In [ ]:
caf['name'].value_counts()

In [ ]:
# remove spaces and accents + make everything same case 
caf['name_cleaned'] = caf['name'].apply(lambda x: unidecode.unidecode(x.strip().title()))

caf['name_cleaned'] = caf['name_cleaned'].apply(lambda x: 'Shawn Oh' if x in ['Shawn Hyup Oh', 'Shawn H. Oh', 'Shawn Oh'] else x)
caf['name_cleaned'] = caf['name_cleaned'].apply(lambda x: 'Ricardo Taboada' if x in ['Ricardo Taboadda'] else x)
caf['name_cleaned'] = caf['name_cleaned'].apply(lambda x: 'Alessandra Gomez' if x in ['Alessandra'] else x)
caf['name_cleaned'] = caf['name_cleaned'].apply(lambda x: 'Charles Beck' if x in ['Cj Beck'] else x)

In [ ]:
pd.crosstab(index=caf['name_cleaned'], columns='count').sort_values('count', ascending=False)

2b. general case info 

In [ ]:
# date - verify all in march-august 2022

caf['month'] = caf['date'].apply(lambda x: x.month)
pd.crosstab(index=caf['month'], columns='count').sort_values('month')

In [ ]:
caf['year'] = caf['date'].apply(lambda x: x.year)
pd.crosstab(index=caf['year'], columns='count').sort_values('year')

In [ ]:
# location 
# pd.crosstab(index=caf['court_division_district'], columns='count').sort_values('count', ascending=False)

# not worth cleaning - verified with naomi that all caf cases are in same location 

2c. judges' info 

In [ ]:
# clean name 

pd.crosstab(index=caf['judge_name'], columns='count').sort_values('count', ascending=False)

In [ ]:
caf['judge_name_cleaned'] = caf['judge_name'].apply(lambda x: 'Nichole Patton' if 'patton' in x.lower()
                                   else ('James Carroll' if 'carroll' in x.lower()
                                   else ('Paul Karkula' if 'karkula' in x.lower()
                                   else x)))
pd.crosstab(index=caf['judge_name_cleaned'], columns='count').sort_values('count', ascending=False)

In [ ]:
pd.crosstab(index=caf['judge_name_cleaned'], columns=caf['judge_race'])

In [ ]:
pd.crosstab(index=caf['judge_name_cleaned'], columns=caf['judge_gender'])

# some weird variations, but seemingly judge carroll = white man, patton = black female, karkula = white man 

2d. jurisdiction info 

In [ ]:
pd.crosstab(index=caf['suburban_district'], columns='count').sort_values('count', ascending=False)

In [ ]:
caf['suburban_district_cleaned'] = caf['suburban_district'].astype(str).apply(lambda x: 'Schiller Park' if 'Schiller Park' in x
                                          else 'Orland Park' if 'Orland Park' in x
                                          else 'Unsure' if 'Not certain' in x
                                          else 'Tinely park' if 'Tenley' in x
                                          else x.replace('Police Department', '').replace(', IL', ''))
pd.crosstab(index=caf['suburban_district_cleaned'], columns='count').sort_values('suburban_district_cleaned')

In [ ]:
pd.crosstab(index=caf['isp_district'], columns='count').sort_values('count', ascending=False)

In [ ]:
pd.crosstab(index=caf['isp_district_cleaned'], columns='count').sort_values('count', ascending=False)

In [ ]:
caf['isp_district_cleaned'] = caf['isp_district'].astype(str).apply(lambda x: 'Unsure' if 'not' in x.lower() or 'Unknown' in x or 'unclear' in x or 'Officier' in x else x)
pd.crosstab(index=caf['isp_district_cleaned'], columns='count').sort_values('count', ascending=False)

In [ ]:
pd.crosstab(index=caf['seizing_entity'], columns='count').sort_values('count', ascending=False)

# 3. clean qualitative/open-ended questions 

3a. property seized 

In [ ]:
caf['seized_property_type_cleaned'] = caf['seized_property_type'].astype(str).apply(lambda x: x.lower().strip())

pd.crosstab(index=caf['seized_property_type_cleaned'], columns='count').sort_values('count', ascending=False)

In [ ]:
caf['more_than_one_asset_explanation_cleaned'] = caf['more_than_one_asset_explanation'].astype(str).apply(lambda x: x.lower().strip())

pd.crosstab(index=caf['more_than_one_asset_explanation_cleaned'], columns='count').sort_values('count', ascending=False)

In [ ]:
caf['property_description_cleaned'] = caf['property_description'].astype(str).apply(lambda x: x.lower().strip())

pd.crosstab(index=caf['property_description_cleaned'], columns='count').sort_values('count', ascending=False)

In [ ]:
# will start coding most common categories here to save some time, but ultimately these will be checked + additional information added manually 

# if any of these key words are in any of the three columns that relate to property, code as that property type 
def property_type(df, word_list, result_if_true):
    if (any(word in df['seized_property_type_cleaned'] for word in word_list) 
        or any(word in df['more_than_one_asset_explanation_cleaned'] for word in word_list) 
        or any(word in df['property_description_cleaned'] for word in word_list)):
        return result_if_true
    else:
        return 0

caf['vehicle_seized'] = caf.apply(property_type, axis=1, args=(['car', 'vehicle', 'chevy', 'mercedes'], 1))
caf['cash_seized'] = caf.apply(property_type, axis=1, args=(['cash', '$', 'money', 'dollar'], 1))
caf['drugs_seized'] = caf.apply(property_type, axis=1, args=(['mj', 'weed', 'cannabis', 'narcotics'], 1))

In [ ]:
# qc 
#caf.loc[caf['vehicle_seized'] == 1, ][['seized_property_type_cleaned', 'more_than_one_asset_explanation_cleaned', 'property_description_cleaned', 'vehicle_seized']]
#caf.loc[caf['cash_seized'] == 1, ][['seized_property_type_cleaned', 'more_than_one_asset_explanation_cleaned', 'property_description_cleaned', 'cash_seized']]
#caf.loc[caf['drugs_seized'] == 1, ][['seized_property_type_cleaned', 'more_than_one_asset_explanation_cleaned', 'property_description_cleaned', 'drugs_seized']]

3b. property value 

In [ ]:
# same note as last time - a lot of these will need to be manually entered and everything will be manually checked
caf['seized_property_value_cleaned'] = caf['seized_property_value'].astype(str).apply(lambda x: x.lower().strip())

pd.crosstab(index=caf['seized_property_value_cleaned'], columns='count').sort_values('count', ascending=False)

In [ ]:
def clean_value(value):
    try: return(int(value))
    except: return(np.nan)
    
caf['seized_property_value_cleaned'] = caf['seized_property_value_cleaned'].apply(clean_value)
# caf[['seized_property_value', 'seized_property_value_cleaned']] #qc

3c. reason for arrest

In [ ]:
caf['seizure_reason_cleaned'] = caf['seizure_reason'].astype(str).apply(lambda x: 'unknown' if x.lower().strip() in ['not specified', 'not sure', 'unsure', 'unclear', 'nan', 'not explained', 'not mentioned'] else x.lower().strip())

pd.crosstab(index=caf['seizure_reason_cleaned'], columns='count').sort_values('count', ascending=False)

In [ ]:
caf['drugs'] = caf['seizure_reason_cleaned'].apply(lambda x: 1 if any(word in x for word in ['narcotics', 'weed', 'marijuana'])  else 0)
caf['aggravated_fleeing_eluding']  = caf['seizure_reason_cleaned'].apply(lambda x: 1 if any(word in x for word in ['flee', 'elude', 'fled'])  else 0)
caf['dui'] = caf['seizure_reason_cleaned'].apply(lambda x: 1 if any(word in x for word in ['influence', 'dui'])  else 0)
caf['driving_without_license'] = caf['seizure_reason_cleaned'].apply(lambda x: 1 if any(word in x for word in ['license'])  else 0)
caf['burglary'] = caf['seizure_reason_cleaned'].apply(lambda x: 1 if any(word in x for word in ['burglary', 'burgle'])  else 0)
caf['robbery'] = caf['seizure_reason_cleaned'].apply(lambda x: 1 if any(word in x for word in ['robbery'])  else 0)

In [ ]:
# qc
# caf.loc[caf['drugs'] == 1, ][['property_seized_cleaned']]

# 4. Export for manual clean-up

In [ ]:
caf.to_excel('caf_pt1_output.xlsx')